a, b, p are given parameters
first construct genPoints.

In [15]:
import random

In [16]:
def finite_field(p):
    return [x for x in range(p)]

In [17]:
def mod_inverse(k, p):
     
    """Returns the inverse of k modulo p.
    This function returns the only integer x such that (x * k) % p == 1.
    k must be non-zero and p must be a prime.
    """
    if k == 0:
        raise ZeroDivisionError('division by zero')

    if k < 0:
        # k ** -1 = p - (-k) ** -1  (mod p)
        return p - mod_inverse(-k, p)

    # Extended Euclidean algorithm.
    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = p, k

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    gcd, x, y = old_r, old_s, old_t

    assert gcd == 1
    assert (k * x) % p == 1

    return x % p

In [18]:
def mod_sqrt(a, p):

    def legendre_symbol(a, p):
        ls = pow(a, (p - 1) // 2, p)
        return -1 if ls == p - 1 else ls
    
    if legendre_symbol(a, p) != 1:
        return 0
    elif a == 0:
        return 0
    elif p == 2:
        return p
    elif p % 4 == 3:
        return pow(a, (p + 1) // 4, p)

    s = p - 1
    e = 0
    while s % 2 == 0:
        s //= 2
        e += 1

    n = 2
    while legendre_symbol(n, p) != -1:
        n += 1

    x = pow(a, (s + 1) // 2, p)
    b = pow(a, s, p)
    g = pow(n, s, p)
    r = e

    while True:
        t = b
        m = 0
        for m in range(r):
            if t == 1:
                break
            t = pow(t, 2, p)

        if m == 0:
            return x

        gs = pow(g, 2 ** (r - m - 1), p)
        g = (gs * gs) % p
        x = (x * gs) % p
        b = (b * g) % p
        r = m

In [19]:
def generate_points(a, b, p):
    points = []
    x = 0
    while(x < p):
        y = ( pow(x, 3) + a*x + b ) % p
        if mod_sqrt(y, p) != 0 and y in finite_field(p):
            points.append((x, mod_sqrt(y, p)))
            points.append((x, -mod_sqrt(y, p) % p))
        x = x + 1
    return points

In [20]:
#returns G
def base_point(p):
    if p[0][1] > p[1][1]:
        return p[1]
    else:
        return p[0]

In [21]:
#returns Pm
def primitive_point():
    return random.choice(affine_points)

In [22]:
#reurns k
def random_generator(p):
    return random.randint(0, p - 1)

In [23]:
#returns P + Q
def point_addition(P, Q, p):
    if P is None:
        # 0 + point2 = point2
        return Q
    if Q is None:
        # point1 + 0 = point1
        return P
    if P[0] == Q[0] and P[1] != Q[1]:
        # point1 + (-point1) = 0
        return None
    num = Q[1] - P[1]
    den = Q[0] - P[0]
    inv_den = mod_inverse(den, p)
    slope = (num * inv_den) % p
    R = [0, 0]
    R[0] = ( pow(slope, 2) - P[0] - Q[0] ) % p
    R[1] = ( slope*(P[0] - R[0]) - P[1] ) % p
    
    return R

In [24]:
#returns 2P
def point_doubling(P, p, a):
    if P is None:
        return P
    if P[1] == 0:
        return None
    num = 3*pow(P[0], 2) + a
    den = 2*P[1]
    inv_den = mod_inverse(den, p)
    slope = (num * inv_den) % p
    R = [0, 0]
    R[0] = ( pow(slope, 2) - 2*P[0] ) % p
    R[1] = ( slope*(P[0] - R[0]) - P[1] ) % p
    
    return R

In [25]:
#returns nP
def scalar_mult(k, P, p, a):
#     """Returns k * point computed using the double and point_add algorithm."""
#     assert is_on_curve(point)

#     if k % curve.n == 0 or point is None:
#         return None

#     if k < 0:
#         # k * point = -k * (-point)
#         return scalar_mult(-k, point_neg(point))

    result = None
    addend = P

    while k:
        if k & 1:
            # Add.
            result = point_addition(result, addend, p)

        # Double.
        addend = point_doubling(addend, p, a)

        k >>= 1

    return result

* G is the base point
* Pm is any random affine point for the given EC
* random integer k, choosen at random < p
* kG is evaluated using addition
* nb is the secret key of host B, generated randomly < p
* Pb is the public key of host B, Pb = nb*G


In [26]:
def ecc_encrypt(G, Pm, k, a, p, char):
    #nb = random_generator(p)
    nb = 17
    Pb = scalar_mult(nb, G, p, a)
    Pml = scalar_mult(ord(char), Pm, p, a)
    kPb = scalar_mult(k, Pb, p, a)
    Pml_kPb = point_addition(Pml, kPb, p)
    kG = scalar_mult(k, G, p, a)
    
    return [ kG, Pml_kPb ]

In [27]:
# Setting up the Elliptic Curve Equation
a = 1
b = 1
p = 37
ecc_points = generate_points(a, b, p)
G = base_point(ecc_points)
#Pm = primitive_point()
Pm = [1, 15]
#k = random_generator(p)
k = 13

encrypted_charachter = ecc_encrypt(G, Pm, k, a, p, "#")
print(encrypted_charachter)

[[0, 1], [30, 24]]


In [37]:
def point_neg(point, p):
    """Returns -point."""
    #assert is_on_curve(point)

    if point is None:
        # -0 = 0
        return None

    x, y = point
    result = (x, -y % p)

    #assert is_on_curve(result)

    return result

In [38]:
def ecc_decrypt(enc_msg, nb, p, a):
    kG = enc_msg[0]
    Pml_kPb = enc_msg[1]
    nbkG = scalar_mult(nb, kG, p, a)
    Pml = point_addition(Pml_kPb, point_neg(nbkG, p), p)
    return Pml

In [39]:
print(ecc_decrypt(encrypted_charachter, 17, 37, 1))

[2, 14]
